In [5]:
import wikipedia
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [51]:
def scrape_wikipedia(category, num_articles):
    article_titles = wikipedia.search(category, results=num_articles)
    articles = []
    for title in article_titles:
        try:
            article = wikipedia.page(title).content
            articles.append(article)
        except wikipedia.exceptions.DisambiguationError as e:
            continue
    return articles

# Scrape Wikipedia articles for each category
category1_articles = scrape_wikipedia("Football", 5)
category2_articles = scrape_wikipedia("Cricket", 5)


C:\Users\91720\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\91720\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [53]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Define a list of stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Add spaces before and after special characters
    text = re.sub(r'[^a-zA-Z \s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Join the words back into a string
    text = ' '.join(words)
    return text

# Preprocess articles for each category
category1_articles = [preprocess_text(article) for article in category1_articles]
category2_articles = [preprocess_text(article) for article in category2_articles]
print(category1_articles)


['football family team sports involve varying degrees kicking ball score goal unqualified word football normally means form football popular word used sports commonly called football include association football known soccer australia canada south africa united states sometimes ireland new zealand australian rules football gaelic football gridiron football specifically american football arena football canadian football international rules football rugby league football rugby union football various forms football share varying degrees common origins known football codes number references traditional ancient prehistoric ball games played many different parts world contemporary codes football traced back codification games english public schools th century outgrowth medieval football expansion cultural power british empire allowed rules football spread areas british influence outside directly controlled empire end th century distinct regional codes already developing gaelic football examp

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91720\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91720\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
# Combine articles from both categories
all_articles = category1_articles + category2_articles

# Initialize CountVectorizer for unigram and bigram counts
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))

# Initialize TfidfVectorizer for TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform unigram and bigram vectorizers
unigram_counts = unigram_vectorizer.fit_transform(all_articles)
bigram_counts = bigram_vectorizer.fit_transform(all_articles)

# Fit and transform TF-IDF vectorizer
tfidf_matrix = tfidf_vectorizer.fit_transform(all_articles)

# print("unigram count matrix: ",unigram_counts)


In [59]:
# Get vocabulary from vectorizers
unigram_vocab = unigram_vectorizer.vocabulary_
bigram_vocab = bigram_vectorizer.vocabulary_

# Invert vocabulary dictionaries to map indices to words
unigram_idx_to_word = {idx: word for word, idx in unigram_vocab.items()}
bigram_idx_to_word = {idx: word for word, idx in bigram_vocab.items()}

# Convert unigram and bigram count matrices to dictionaries
unigram_counts_dict = {}
for i, article in enumerate(all_articles):
    for j, idx in enumerate(unigram_vocab.values()):
        word = unigram_idx_to_word[idx]
        count = unigram_counts[i, j]
        if word in unigram_counts_dict:
            unigram_counts_dict[word] += count
        else:
            unigram_counts_dict[word] = count

bigram_counts_dict = {}
for i, article in enumerate(all_articles):
    for j, idx in enumerate(bigram_vocab.values()):
        word = bigram_idx_to_word[idx]
        count = bigram_counts[i, j]
        if word in bigram_counts_dict:
            bigram_counts_dict[word] += count
        else:
            bigram_counts_dict[word] = count

# Sort dictionaries by frequency in descending order
sorted_unigram_counts = sorted(unigram_counts_dict.items(), key=lambda x: x[1], reverse=True)
sorted_bigram_counts = sorted(bigram_counts_dict.items(), key=lambda x: x[1], reverse=True)

# Print top 10 unigrams and bigrams
print("Top 10 Unigrams:")
for word, freq in sorted_unigram_counts[:15]:
    print(f"{word}: {freq}")

print("\nTop 10 Bigrams:")
for word, freq in sorted_bigram_counts[:15]:
    print(f"{word}: {freq}")


Top 10 Unigrams:
rutgers: 812
ship: 552
proposed: 466
need: 466
leaves: 329
ranking: 293
apparent: 292
supplemental: 288
career: 262
officer: 250
guinea: 242
verb: 230
specialists: 229
bbc: 213
wealthiest: 197

Top 10 Bigrams:
variations rules: 179
school tasked: 87
remain different: 66
doubtful win: 65
eventually five: 63
years canada: 59
stands laying: 52
zealand south: 42
cricket one: 40
quoted saying: 38
attendance increased: 35
ii devastating: 35
individuals accounting: 32
italics last: 30
levels sports: 29


In [60]:
from sklearn.model_selection import train_test_split

# Combine articles and labels
articles = category1_articles + category2_articles
labels = [0] * len(category1_articles) + [1] * len(category2_articles)

# Split data into training and testing sets
train_articles, test_articles, train_labels, test_labels = train_test_split(articles, labels, test_size=0.2, random_state=42)

# Initialize vectorizers
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform vectorizers
train_unigram = unigram_vectorizer.fit_transform(train_articles)
test_unigram = unigram_vectorizer.transform(test_articles)

train_bigram = bigram_vectorizer.fit_transform(train_articles)
test_bigram = bigram_vectorizer.transform(test_articles)

train_tfidf = tfidf_vectorizer.fit_transform(train_articles)
test_tfidf = tfidf_vectorizer.transform(test_articles)

# Initialize Naive Bayes classifiers
nb_unigram = MultinomialNB()
nb_bigram = MultinomialNB()
nb_tfidf = MultinomialNB()

# Train classifiers
nb_unigram.fit(train_unigram, train_labels)
nb_bigram.fit(train_bigram, train_labels)
nb_tfidf.fit(train_tfidf, train_labels)

# Predict labels for testing set
pred_unigram = nb_unigram.predict(test_unigram)
pred_bigram = nb_bigram.predict(test_bigram)
pred_tfidf = nb_tfidf.predict(test_tfidf)

# Evaluate accuracy of classifiers
acc_unigram = accuracy_score(test_labels, pred_unigram)
acc_bigram = accuracy_score(test_labels, pred_bigram)
acc_tfidf = accuracy_score(test_labels, pred_tfidf)

print("Accuracy using unigram counts:", acc_unigram)
print("Accuracy using bigram counts:", acc_bigram)
print("Accuracy using TF-IDF matrix:", acc_tfidf)


Accuracy using unigram counts: 1.0
Accuracy using bigram counts: 1.0
Accuracy using TF-IDF matrix: 1.0


In [71]:
# Scrape a Wikipedia article for unseen data
unseen_articles = scrape_wikipedia("Football Association", 1)
if unseen_articles:
    unseen_article = unseen_articles[0]

    # Preprocess unseen article
    preprocessed_unseen_article = preprocess_text(unseen_article)

    # Transform unseen article using vectorizers
    unseen_unigram = unigram_vectorizer.transform([preprocessed_unseen_article])
    unseen_bigram = bigram_vectorizer.transform([preprocessed_unseen_article])
    unseen_tfidf = tfidf_vectorizer.transform([preprocessed_unseen_article])

    # Predict label for unseen data
    pred_unseen_unigram = nb_unigram.predict(unseen_unigram)
    pred_unseen_bigram = nb_bigram.predict(unseen_bigram)
    pred_unseen_tfidf = nb_tfidf.predict(unseen_tfidf)

    print("Predicted label for unseen article (unigram):", pred_unseen_unigram)
    print("Predicted label for unseen article (bigram):", pred_unseen_bigram)
    print("Predicted label for unseen article (TF-IDF):", pred_unseen_tfidf)
else:
    print("No articles found for scraping.")


Predicted label for unseen article (unigram): [0]
Predicted label for unseen article (bigram): [0]
Predicted label for unseen article (TF-IDF): [0]


In [72]:
# # True labels for unseen data
# true_label_unseen_category1 = 0
# true_label_unseen_category2 = 1

# # Compute accuracy for unigram
# accuracy_unseen_unigram = (pred_unseen_unigram[0] == true_label_unseen_category1) and (pred_unseen_category2_unigram[0] == true_label_unseen_category2)
# accuracy_unseen_unigram = accuracy_unseen_unigram * 100  # Convert to percentage

# # Compute accuracy for bigram
# accuracy_unseen_bigram = (pred_unseen_bigram[0] == true_label_unseen_category1) and (pred_unseen_category2_bigram[0] == true_label_unseen_category2)
# accuracy_unseen_bigram = accuracy_unseen_bigram * 100  # Convert to percentage

# # Compute accuracy for TF-IDF
# accuracy_unseen_tfidf = (pred_unseen_tfidf[0] == true_label_unseen_category1) and (pred_unseen_category2_tfidf[0] == true_label_unseen_category2)
# accuracy_unseen_tfidf = accuracy_unseen_tfidf * 100  # Convert to percentage

# print("Accuracy on unseen data (unigram):", accuracy_unseen_unigram)
# print("Accuracy on unseen data (bigram):", accuracy_unseen_bigram)
# print("Accuracy on unseen data (TF-IDF):", accuracy_unseen_tfidf)
